In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [87]:
# load locally saved results of gridsearch
results = pd.read_pickle("/home/tom/local_data/xgboost_uk_region_grid_search_results.p")
best_params = results.sort_values("NegMAE", ascending=False).iloc[0]  # best params

# recast some params to float
best_params.loc[["gamma", "reg_alpha", "reg_lambda"]] = best_params.loc[["gamma", "reg_alpha", "reg_lambda"]].astype(float)

In [88]:
best_params

colsample_bytree         0.75
gamma                     0.0
grow_policy         depthwise
learning_rate            0.01
max_depth                 300
min_child_weight            5
n_estimators             2000
reg_alpha                 0.0
reg_lambda                0.0
subsample                0.95
NegMAE              -0.021343
Name: 866, dtype: object

In [114]:
# for each param, do a finer grid search
# consider other params fixed during search to reduce the time taken

DEFAULT_PERCENTILES = np.asarray([0.75, 0.85, 1.25, 1.4])
DEFFAULT_HYPARAM_CONFIG = {
    "objective": "reg:squarederror",
    "booster": "gbtree",
    "colsample_bylevel": 1,
    "colsample_bynode": 1,
    "colsample_bytree": 0.85,
    "early_stopping_rounds": None,
    "gamma": 0,
    "gpu_id": -1,
    "grow_policy": "depthwise",
    "importance_type": None,
    "interaction_constraints": "",
    "learning_rate": 0.005,
    "max_bin": 256,
    "max_cat_threshold": 64,
    "max_depth": 100,
    "max_leaves": 0,
    "min_child_weight": 5,
    "n_estimators": 1_500,
    "n_jobs": -1,
    "num_parallel_tree": 1,
    "predictor": "auto",
    "random_state": 0,
    "reg_alpha": 0,
    "reg_lambda": 1,
    "sampling_method": "uniform",
    "scale_pos_weight": 1,
    "subsample": 0.65,
    "tree_method": "hist",
    "validate_parameters": 1,
    "verbosity": 1,
}


def create_param_grids( current_params: dict[str, float], percentiles:np.ndarray = DEFAULT_PERCENTILES):
    grid = list()
    for param, param_value in current_params.items():
        if isinstance(param_value, str):
            pass
        else:
            if np.isclose(param_value, 0, rtol=1e-6):
                grid.append(pd.Series(index=range(len(percentiles)), data =np.linspace(0, 2, len(DEFAULT_PERCENTILES)), name=param))
            else:
                grid.append(pd.Series(index=range(len(percentiles)), data = DEFAULT_PERCENTILES*param_value, name=param))
        
    return pd.concat(grid, axis=1)

def get_default_hyperparam_config() -> dict:
    return DEFFAULT_HYPARAM_CONFIG


In [110]:
# load in data for running experiment
import numpy as np
import pandas as pd
import xarray as xr
from pandas.core.dtypes.common import is_datetime64_dtype
import numpy.typing as npt
from typing import Union, Tuple, List

nwp_path = (
    "gs://solar-pv-nowcasting-data/NWP/UK_Met_Office/UKV_intermediate_version_3.zarr/"
)
gsp_path = "gs://solar-pv-nowcasting-data/PV/GSP/v5/pv_gsp.zarr"

TRIG_DATETIME_FEATURE_NAMES = [
    "SIN_MONTH",
    "COS_MONTH",
    "SIN_DAY",
    "COS_DAY",
    "SIN_HOUR",
    "COS_HOUR",
]


def _get_path_to_uk_region_data_data(
    variable: str, forecast_horizon: int, inner: bool
) -> str:
    if inner:
        return f"/home/tom/local_data/uk_region_mean_var{variable}_step{forecast_horizon}.npy"
    else:
        return f"/home/tom/local_data/outer_region_mean_var{variable}_step{forecast_horizon}.npy"


def _trig_transform(
    values: np.ndarray, period: Union[float, int]
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Given a list of values and an upper limit on the values, compute trig decomposition.
    Args:
        values: ndarray of points in the range [0, period]
        period: period of the data
    Returns:
        Decomposition of values into sine and cosine of data with given period
    """

    return np.sin(values * 2 * np.pi / period), np.cos(values * 2 * np.pi / period)


def trigonometric_datetime_transformation(datetimes: npt.ArrayLike) -> np.ndarray:
    """
    Given an iterable of datetimes, returns a trigonometric decomposition on hour, day and month
    Args:init_time=30,
        datetimes: ArrayLike of datetime64 values
    Returns:
        Trigonometric decomposition of datetime into hourly, daily and
        monthly values.
    """
    assert is_datetime64_dtype(
        datetimes
    ), "Data for Trig Decomposition must be np.datetime64 type"

    datetimes = pd.DatetimeIndex(datetimes)
    hour = datetimes.hour.values.reshape(-1, 1) + (
        datetimes.minute.values.reshape(-1, 1) / 60
    )
    day = datetimes.day.values.reshape(-1, 1)
    month = datetimes.month.values.reshape(-1, 1)

    sine_hour, cosine_hour = _trig_transform(hour, 24)
    sine_day, cosine_day = _trig_transform(day, 366)
    sine_month, cosine_month = _trig_transform(month, 12)

    return np.concatenate(
        [sine_month, cosine_month, sine_day, cosine_day, sine_hour, cosine_hour], axis=1
    )


DEFAULT_UK_REGION_NWP_VARS = ["dswrf", "hcct", "t", "lcc", "sde"]


def build_datasets_from_local(
        eval_timeseries: pd.DatetimeIndex,
        step: int,
        variables: list[str] = DEFAULT_UK_REGION_NWP_VARS,
        nan_to_zero: bool = False,
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:

    X = list()
    for var in variables:
        X.append(np.load(_get_path_to_uk_region_data_data(var, step, True)))
        X.append(np.load(_get_path_to_uk_region_data_data(var, step, False)))
    X = np.concatenate(X, axis=0).T
    X = np.nan_to_num(X) if nan_to_zero else X

    columns = []
    for var in variables:
        columns += [f"{var}_{region}" for region in ["within", "outer"]]

    X = pd.DataFrame(data=X, columns=columns, index=eval_timeseries)
    y = pd.DataFrame(
        gsp["generation_mw"] / gsp["installedcapacity_mwp"],
        index=eval_timeseries,
        columns=["target"],
    )

    # shift y by the step forecast
    shift = nwp.step.values[step]
    y = y.shift(freq=-shift).dropna()
    common_index = sorted(pd.DatetimeIndex((set(y.index).intersection(X.index))))

    X, y = X.loc[common_index], y.loc[common_index]

    # add datetime methods for the point at which we are forecasting e.g. now + step
    _X = trigonometric_datetime_transformation(
        y.shift(freq=nwp.step.values[step]).index.values
    )
    _X = pd.DataFrame(_X, index=y.index, columns=TRIG_DATETIME_FEATURE_NAMES)
    X = pd.concat([X, _X], axis=1)

    # add lagged values of GSP PV
    ar_1 = y.shift(freq=-(shift + np.timedelta64(1, "h")))
    ar_day = y.shift(freq=-(shift + np.timedelta64(1, "D")))
    ar_1.columns = ["PV_LAG_1HR"]
    ar_day.columns = ["PV_LAG_DAY"]

    # estimate linear trend of the PV
    window_size = 10
    epsilon = 0.01
    y_covariates = y.shift(freq=-(shift + np.timedelta64(2, "h")))
    y_covariates.columns = ["x"]
    y_target = y.shift(freq=-(shift + np.timedelta64(1, "h")))
    y_target.columns = ["y"]
    data = pd.concat([y_target, y_covariates], axis=1).dropna()
    _x = data["x"].values
    _y = data["y"].values
    _betas = np.nan * np.empty(len(data))

    for n in range(window_size, len(data)):
        __y = _y[(n - window_size) : n]
        __x = _x[(n - window_size) : n]
        __b = max(min((1 / ((__x.T @ __x) + epsilon)) * (__x.T @ __y), 10), -10)
        _betas[n] = __b

    betas = pd.DataFrame(data=_betas, columns=["AR_Beta"], index=data.index)

    X = pd.concat([X, ar_1, ar_day, betas], axis=1).dropna()
    y = y.loc[X.index]

    return X, y
    
def build_ts_data_cv_splitting(
    X: pd.DataFrame, n_splits: int, val_size: int
) -> List[Tuple]:
    X_tests = range(0, len(X) - val_size, int((len(X) - val_size) / n_splits))
    prev_idx = 0
    indicies = list()

    for idx in X_tests[1:]:
        indicies.append((list(range(prev_idx, idx)), list(range(idx, idx + val_size))))

    return indicies

In [111]:
gsp = xr.open_zarr(gsp_path)
nwp = xr.open_zarr(nwp_path)

evaluation_timeseries = (
    gsp.coords["datetime_gmt"]
    .where(
        (gsp["datetime_gmt"] >= nwp.coords["init_time"].values[0])
        & (gsp["datetime_gmt"] <= nwp.coords["init_time"].values[-1]),
        drop=True,
    )
    .values
)

gsp = gsp.sel(datetime_gmt=evaluation_timeseries, gsp_id=0)
X, y = build_datasets_from_local(
    evaluation_timeseries, 30
)  # choose forecast horizon 30 for the CV
_cv = build_ts_data_cv_splitting(X, 3, 2_000)

In [112]:
best_params.to_dict()["n_estimators"].__class__

int

In [130]:
# best_params.drop("NegMAE", inplace=True)

param_search_grid = create_param_grids(best_params.to_dict())
param_search_grid[["colsample_bytree", "subsample"]] = param_search_grid[["colsample_bytree", "subsample"]].clip(0, 1)
param_search_grid[["max_depth", "min_child_weight", "n_estimators"]] = param_search_grid[["max_depth", "min_child_weight", "n_estimators"]].astype(np.int64)
param_search_grid = param_search_grid.to_dict(orient="list")
param_search_grid

{'colsample_bytree': [0.5625, 0.6375, 0.9375, 1.0],
 'gamma': [0.0, 0.6666666666666666, 1.3333333333333333, 2.0],
 'learning_rate': [0.0075, 0.0085, 0.0125, 0.013999999999999999],
 'max_depth': [225, 255, 375, 420],
 'min_child_weight': [3, 4, 6, 7],
 'n_estimators': [1500, 1700, 2500, 2800],
 'reg_alpha': [0.0, 0.6666666666666666, 1.3333333333333333, 2.0],
 'reg_lambda': [0.0, 0.6666666666666666, 1.3333333333333333, 2.0],
 'subsample': [0.7124999999999999, 0.8075, 1.0, 1.0]}

In [84]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

def run_grid_search_experiment(default_xgboost_params, gsearch_params):
    gsearch = GridSearchCV(
        XGBRegressor(**default_xgboost_params),
        param_grid=gsearch_params,
        scoring="neg_mean_absolute_error",
        verbose=1,
        cv=_cv
    )
    
    results = gsearch.fit(X, y)
    scores = pd.concat(
        [
            pd.DataFrame(results.cv_results_["params"]),
            pd.DataFrame(results.cv_results_["mean_test_score"], columns=["NegMAE"]),
        ],
        axis=1,
    )
    
    return scores
    

In [131]:
output = dict()

for param in param_search_grid:
    default_hyperparams = get_default_hyperparam_config()
    del default_hyperparams[param]  # remove param from default parameters
    
    search = {
        param: param_search_grid[param]
    }
    
    output[param] = run_grid_search_experiment(default_hyperparams, search)
    
     

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits


In [148]:
optimal_params = dict()

for param, x in output.items():
    optimal_params[param] = x.sort_values("NegMAE", ascending=False).iloc[0][param]

In [149]:
optimal_params

{'colsample_bytree': 0.6375,
 'gamma': 0.6666666666666666,
 'learning_rate': 0.0075,
 'max_depth': 225.0,
 'min_child_weight': 6.0,
 'n_estimators': 2500.0,
 'reg_alpha': 0.6666666666666666,
 'reg_lambda': 2.0,
 'subsample': 1.0}